In [1]:
!make docker-image > /dev/null 2>&1

In [2]:
!CONTAINER_CMD="bash -lc 'make install-ycsb" make docker > /dev/null 2>&1

In [3]:
from pathlib import Path
import pexpect
import os
import time

""" Collector class has helper methods to interact with kermit"""
class Collector: 
    def __init__(self, config: Path):
        self.env = os.environ.copy()
        self.env["INTERACTIVE"] = "it"
        self.env["CONTAINER_CMD"] = f"bash -lc 'KERNMLOPS_CONFIG_FILE={config} make collect-data'"
        self.collect_process : pexpect.spawn | None = None

    def start_collection(self, logfile=None):
        self.collect_process = pexpect.spawn("make docker", env=self.env, timeout=None, logfile=logfile)
        self.collect_process.expect_exact(["Started benchmark"])

    def _after_run_generate_file_data() -> dict[str, list[Path]]:
        start_path : Path = Path("./data")
        list_of_collect_id_dirs = start_path.glob("*/*/*")
        latest_collect_id = max(list_of_collect_id_dirs, key=os.path.getctime)
        list_of_files = latest_collect_id.glob("*.*.parquet")
        output = {}
        for f in list_of_files:
            index = str(f).removeprefix(str(f.parent) + "/").split(".")[0]
            if index not in output.keys():
                output[index] = []
            output[index].append(f)
        return output
        
    def wait(self) -> int:
        if self.collect_process is None:
            return
        self.collect_process.expect([pexpect.EOF])
        self.collect_process.wait()
        return Collector._after_run_generate_file_data()
        
    def stop_collection(self):
        if self.collect_process is None:
            return
        self.collect_process.sendline("END")
        return self.wait()

There are two ways to run kermit:
- With the raw config
- With a pre-programmed benchmark config

In [4]:
import subprocess
import sys


# New TCP Collector
collect = Collector("./config/raw_overrides.yaml")
collect.start_collection()

# Generate some TCP traffic
!nc -l 8080 &  # Listen on port 8080
!echo "testtesttesttesttesttesttesttesttesttest" | nc localhost 8080 
!echo "testtesttesttesttesttesttesttesttesttest" | nc localhost 8080 
!echo "testtesttesttesttesttesttesttesttesttest" | nc localhost 8080 
!echo "testtesttesttesttesttesttesttesttesttest" | nc localhost 8080 
!echo "testtesttesttesttesttesttesttesttesttest" | nc localhost 8080 
!echo "testtesttesttesttesttesttesttesttesttest" | nc localhost 8080 

data = collect.stop_collection()

In [5]:
data

{}

In [6]:
# Analyze TCP branches
import polars as pl
tcp_df = pl.read_parquet(data["tcp_v4_rcv"][0])
tcp_state_df = pl.read_parquet(data["tcp_state_process"][0])

print(tcp_df.group_by("comm").count().sort("count", descending=True))
print(tcp_state_df.group_by("comm").count().sort("count", descending=True))

nc = tcp_df.filter(pl.col("comm").str.contains("nc"))
nc_state = tcp_state_df.filter(pl.col("comm").str.contains("nc"))

# Show branch distribution
print(nc.group_by("branch_name").count().sort("count", descending=True))
print(nc_state.group_by("event_type_name").count().sort("count", descending=True))

# Show drop reasons
drops = nc.filter(pl.col("drop_reason") > 0)
print(drops.group_by("drop_reason_name").count())

KeyError: 'tcp_v4_rcv'

In [ ]:
# Cell 1: Load and prepare data
import polars as pl
import matplotlib.pyplot as plt
import numpy as np

# Load the data (assuming data dict is already populated from previous cells)
tcp_df = pl.read_parquet(data["tcp_v4_rcv"][0])
tcp_state_df = pl.read_parquet(data["tcp_state_process"][0])

print(f"Loaded {len(tcp_df):,} tcp_v4_rcv events")
print(f"Loaded {len(tcp_state_df):,} tcp_state_process events")

# Cell 2: Basic exploration of TCP state data
# Show sample of state transitions
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
print("\nSample State Transitions:")
print(transitions.select(['comm', 'old_state_name', 'new_state_name', 'ts_uptime_us']).head(10))

# Show event type distribution
print("\nEvent Type Distribution:")
print(tcp_state_df.group_by('event_type_name').len().sort('len', descending=True))

# Cell 3: Deep dive into specific process (e.g., nc or iperf3)
# Filter for a specific process
process_name = "nc"  # or "iperf3" depending on your test
process_events = tcp_state_df.filter(pl.col('comm').str.contains(process_name))

if len(process_events) > 0:
    print(f"\n{process_name} TCP State Analysis:")
    print(f"Total events: {len(process_events)}")
    
    # Show state progression
    print("\nState progression timeline:")
    timeline = process_events.select(['ts_uptime_us', 'event_type_name', 'old_state_name', 'new_state_name']).sort('ts_uptime_us')
    
    # Convert to relative time in milliseconds
    start_time = timeline['ts_uptime_us'].min()
    timeline_with_relative = timeline.with_columns([
        ((pl.col('ts_uptime_us') - start_time) / 1000).alias('relative_ms')
    ])
    
    # Show key events
    for row in timeline_with_relative.head(20).iter_rows():
        if row[1] == 'TRANSITION':
            print(f"{row[4]:8.1f}ms: {row[2]} → {row[3]}")
        else:
            print(f"{row[4]:8.1f}ms: {row[1]} in {row[3]}")

# Cell 4: Connection lifecycle analysis
print("\n" + "="*50)
print("CONNECTION LIFECYCLE ANALYSIS")
print("="*50)

# Group events by process and analyze connection patterns
connection_patterns = tcp_state_df.group_by('comm').agg([
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'SYN_SENT').len().alias('syn_sent_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'SYN_RECV').len().alias('syn_recv_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'ESTABLISHED').len().alias('established_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'TIME_WAIT').len().alias('time_wait_count'),
    pl.col('event_type_name').filter(pl.col('event_type_name') == 'ERROR').len().alias('error_count')
]).filter(pl.col('established_count') > 0).sort('established_count', descending=True)

print("Connection patterns by process:")
print(connection_patterns.head(10))

# Cell 5: Visualize state machine flow
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# State transition graph
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
if len(transitions) > 0:
    # Count transitions
    trans_counts = transitions.group_by(['old_state_name', 'new_state_name']).len().sort('len', descending=True)
    
    # Create a simple flow diagram representation
    states = set()
    for row in trans_counts.iter_rows():
        states.add(row[0])
        states.add(row[1])
    
    # Plot top transitions as a bar chart
    trans_labels = [f"{row[0]} → {row[1]}" for row in trans_counts.head(15).iter_rows()]
    trans_values = [row[2] for row in trans_counts.head(15).iter_rows()]
    
    ax1.barh(range(len(trans_labels)), trans_values)
    ax1.set_yticks(range(len(trans_labels)))
    ax1.set_yticklabels(trans_labels)
    ax1.set_xlabel('Count')
    ax1.set_title('Top 15 State Transitions')
    ax1.grid(axis='x', alpha=0.3)

# Event timeline
# Create time-based event density
start_time = tcp_state_df['ts_uptime_us'].min()
tcp_state_df_time = tcp_state_df.with_columns([
    ((pl.col('ts_uptime_us') - start_time) / 1_000_000).alias('time_seconds')
])

# Create histogram of events over time
time_bins = np.linspace(0, tcp_state_df_time['time_seconds'].max(), 50)
transitions_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'TRANSITION')['time_seconds'].to_numpy()
processing_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'PROCESSING')['time_seconds'].to_numpy()
errors_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'ERROR')['time_seconds'].to_numpy()

ax2.hist([transitions_time, processing_time, errors_time], 
         bins=time_bins, label=['Transitions', 'Processing', 'Errors'], 
         stacked=True, alpha=0.7)
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Event Count')
ax2.set_title('TCP State Events Over Time')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Cell 6: Correlate with tcp_v4_rcv data
print("\n" + "="*50)
print("CORRELATION WITH TCP_V4_RCV BRANCH DATA")
print("="*50)

# Find overlapping time windows
tcp_v4_time_range = (tcp_df['ts_uptime_us'].min(), tcp_df['ts_uptime_us'].max())
tcp_state_time_range = (tcp_state_df['ts_uptime_us'].min(), tcp_state_df['ts_uptime_us'].max())

overlap_start = max(tcp_v4_time_range[0], tcp_state_time_range[0])
overlap_end = min(tcp_v4_time_range[1], tcp_state_time_range[1])

if overlap_start < overlap_end:
    print(f"Time overlap: {(overlap_end - overlap_start) / 1_000_000:.2f} seconds")
    
    # Filter both datasets to overlapping time
    tcp_v4_overlap = tcp_df.filter(
        (pl.col('ts_uptime_us') >= overlap_start) & 
        (pl.col('ts_uptime_us') <= overlap_end)
    )
    tcp_state_overlap = tcp_state_df.filter(
        (pl.col('ts_uptime_us') >= overlap_start) & 
        (pl.col('ts_uptime_us') <= overlap_end)
    )
    
    # Analyze patterns
    print(f"\nDuring overlap period:")
    print(f"- TCP v4 receive events: {len(tcp_v4_overlap):,}")
    print(f"- TCP state events: {len(tcp_state_overlap):,}")
    
    # Show branch distribution during state transitions
    print("\nTCP v4 branches during state processing:")
    branch_dist = tcp_v4_overlap.group_by('branch_name').len().sort('len', descending=True)
    for row in branch_dist.head(10).iter_rows():
        print(f"  {row[0]:25}: {row[1]:6,} ({row[1]/len(tcp_v4_overlap)*100:5.1f}%)")

# Cell 7: Performance insights
print("\n" + "="*50)
print("PERFORMANCE INSIGHTS")
print("="*50)

# Calculate metrics
total_time = (tcp_state_df['ts_uptime_us'].max() - tcp_state_df['ts_uptime_us'].min()) / 1_000_000
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
errors = tcp_state_df.filter(pl.col('event_type_name') == 'ERROR')

# Connection establishment rate
new_connections = transitions.filter(pl.col('new_state_name') == 'ESTABLISHED')
connection_rate = len(new_connections) / total_time if total_time > 0 else 0

# Error rate
error_rate = len(errors) / len(tcp_state_df) * 100 if len(tcp_state_df) > 0 else 0

# State distribution
state_dist = tcp_state_df.filter(pl.col('event_type_name') == 'PROCESSING').group_by('new_state_name').len()

print(f"Collection duration: {total_time:.2f} seconds")
print(f"Connection establishment rate: {connection_rate:.1f} connections/second")
print(f"Error rate: {error_rate:.2f}%")
print(f"Total state transitions: {len(transitions):,}")

print("\nState distribution (PROCESSING events):")
for row in state_dist.sort('len', descending=True).iter_rows():
    print(f"  {row[0]:15}: {row[1]:6,} events")

# Identify potential issues
print("\nPotential Issues:")
if error_rate > 1:
    print(f"⚠️  High error rate detected: {error_rate:.2f}%")
    
challenge_acks = tcp_state_df.filter(pl.col('event_subtype_name') == 'CHALLENGE_ACK')
if len(challenge_acks) > 0:
    print(f"⚠️  Challenge ACKs detected: {len(challenge_acks)} (possible SYN flood)")
    
resets = tcp_state_df.filter(pl.col('event_subtype_name') == 'RESET')
if len(resets) > 0:
    print(f"⚠️  Connection resets: {len(resets)}")

# Check for connection leaks
time_wait_trans = transitions.filter(pl.col('new_state_name') == 'TIME_WAIT')
close_trans = transitions.filter(pl.col('new_state_name').str.contains('CLOSE'))
if len(new_connections) > 0:
    close_ratio = (len(time_wait_trans) + len(close_trans)) / len(new_connections)
    if close_ratio < 0.8:
        print(f"⚠️  Possible connection leak: only {close_ratio*100:.1f}% of connections properly closed")

In [ ]:
# Simple iperf3 Test - Docker Friendly Version

import subprocess
import time

# First, let's manually test if iperf3 works
print("Testing iperf3 installation...")

# Install iperf3 if needed
install_result = subprocess.run(["which", "iperf3"], capture_output=True)
if install_result.returncode != 0:
    print("Installing iperf3...")
    subprocess.run(["apt-get", "update"], capture_output=True)
    subprocess.run(["apt-get", "install", "-y", "iperf3"], capture_output=True)
    print("iperf3 installed!")
else:
    print("iperf3 is already installed")

# Kill any existing iperf3 processes
subprocess.run(["pkill", "-f", "iperf3"], capture_output=True)
time.sleep(1)

# Start iperf3 server manually
print("\nStarting iperf3 server on port 5555...")
server = subprocess.Popen(
    ["iperf3", "-s", "-p", "5555"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server
time.sleep(3)

# Test connection
print("Testing server connection...")
test = subprocess.run(
    ["iperf3", "-c", "127.0.0.1", "-p", "5555", "-t", "1"],
    capture_output=True,
    text=True
)

if test.returncode == 0:
    print("✓ iperf3 is working!")
    
    # Now run actual benchmark with kernmlops
    print("\nRunning benchmark with kernmlops...")
    
    # Use minimal config for Docker
    collect = Collector("./config/iperf_docker.yaml")
    
    try:
        collect.start_collection(None)
        data = collect.wait()
        
        import polars as pl
        tcp_df = pl.read_parquet(data["tcp_v4_rcv"][0])
        print(f"\n✓ Success! Captured {len(tcp_df):,} TCP events")
        
    except Exception as e:
        print(f"\n✗ Benchmark failed: {e}")
        print("\nTry using the alternative method below...")
        
else:
    print("✗ iperf3 server test failed")
    print(f"Error: {test.stderr}")

# Cleanup
server.terminate()
subprocess.run(["pkill", "-f", "iperf3"], capture_output=True)


In [ ]:
# Analyze by process
print(tcp_df.group_by("comm").count().sort("count", descending=True))
iperf_client = tcp_df.filter(pl.col("comm").str.contains("iperf3").and_(~pl.col("comm").str.contains("-s")))

print(f"\nProcess breakdown:")
print(f"- iperf3 client: {len(iperf_client)} events")

# Analyze port 5555 traffic (iperf3 default port)
port_5555 = tcp_df.filter((pl.col("dport") == 5555) | (pl.col("sport") == 5555))
print(f"- Port 5555 traffic: {len(port_5555)} events")

# Branch distribution
print("\nTCP state distribution:")
branch_dist = tcp_df.group_by("branch_name").agg([
    pl.count().alias("count"),
    (pl.count() / len(tcp_df) * 100).alias("percentage")
]).sort("count", descending=True)

for row in branch_dist.head(10).iter_rows():
    print(f"- {row[0]}: {row[1]:,} events ({row[2]:.1f}%)")

# Connection analysis
new_connections = tcp_df.filter(pl.col("branch_name") == "new_syn_recv")
print(f"\nNew TCP connections: {len(new_connections)}")
print(f"Connections per stream: ~{len(new_connections) / 4:.0f}")  # 4 parallel streams

# Drop analysis
drops = tcp_df.filter(pl.col("drop_reason") > 0)
if len(drops) > 0:
    print(f"\nDropped packets: {len(drops)}")
    drop_dist = drops.group_by("drop_reason_name").len()
    for row in drop_dist.iter_rows():
        print(f"- {row[0]}: {row[1]} drops")
else:
    print("\nNo dropped packets detected - excellent!")

# Show branch distribution
print("iperf_client group by branch_name")
print(iperf_client.group_by("branch_name").count().sort("count", descending=True))

# Show drop reasons
drops = iperf_client.filter(pl.col("drop_reason") > 0)
print(drops.group_by("drop_reason_name").count())

In [ ]:
tcp_state_df = pl.read_parquet(data["tcp_state_process"][0])

print(f"Loaded {len(tcp_df):,} tcp_v4_rcv events")
print(f"Loaded {len(tcp_state_df):,} tcp_state_process events")

# Cell 2: Basic exploration of TCP state data
# Show sample of state transitions
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
print("\nSample State Transitions:")
print(transitions.select(['comm', 'old_state_name', 'new_state_name', 'ts_uptime_us']).head(10))

# Show event type distribution
print("\nEvent Type Distribution:")
print(tcp_state_df.group_by('event_type_name').len().sort('len', descending=True))

# Cell 3: Deep dive into specific process (e.g., nc or iperf3)
# Filter for a specific process
process_name = "nc"  # or "iperf3" depending on your test
process_events = tcp_state_df.filter(pl.col('comm').str.contains(process_name))

if len(process_events) > 0:
    print(f"\n{process_name} TCP State Analysis:")
    print(f"Total events: {len(process_events)}")
    
    # Show state progression
    print("\nState progression timeline:")
    timeline = process_events.select(['ts_uptime_us', 'event_type_name', 'old_state_name', 'new_state_name']).sort('ts_uptime_us')
    
    # Convert to relative time in milliseconds
    start_time = timeline['ts_uptime_us'].min()
    timeline_with_relative = timeline.with_columns([
        ((pl.col('ts_uptime_us') - start_time) / 1000).alias('relative_ms')
    ])
    
    # Show key events
    for row in timeline_with_relative.head(20).iter_rows():
        if row[1] == 'TRANSITION':
            print(f"{row[4]:8.1f}ms: {row[2]} → {row[3]}")
        else:
            print(f"{row[4]:8.1f}ms: {row[1]} in {row[3]}")

# Cell 4: Connection lifecycle analysis
print("\n" + "="*50)
print("CONNECTION LIFECYCLE ANALYSIS")
print("="*50)

# Group events by process and analyze connection patterns
connection_patterns = tcp_state_df.group_by('comm').agg([
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'SYN_SENT').len().alias('syn_sent_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'SYN_RECV').len().alias('syn_recv_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'ESTABLISHED').len().alias('established_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'TIME_WAIT').len().alias('time_wait_count'),
    pl.col('event_type_name').filter(pl.col('event_type_name') == 'ERROR').len().alias('error_count')
]).filter(pl.col('established_count') > 0).sort('established_count', descending=True)

print("Connection patterns by process:")
print(connection_patterns.head(10))

# Cell 5: Visualize state machine flow
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# State transition graph
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
if len(transitions) > 0:
    # Count transitions
    trans_counts = transitions.group_by(['old_state_name', 'new_state_name']).len().sort('len', descending=True)
    
    # Create a simple flow diagram representation
    states = set()
    for row in trans_counts.iter_rows():
        states.add(row[0])
        states.add(row[1])
    
    # Plot top transitions as a bar chart
    trans_labels = [f"{row[0]} → {row[1]}" for row in trans_counts.head(15).iter_rows()]
    trans_values = [row[2] for row in trans_counts.head(15).iter_rows()]
    
    ax1.barh(range(len(trans_labels)), trans_values)
    ax1.set_yticks(range(len(trans_labels)))
    ax1.set_yticklabels(trans_labels)
    ax1.set_xlabel('Count')
    ax1.set_title('Top 15 State Transitions')
    ax1.grid(axis='x', alpha=0.3)

# Event timeline
# Create time-based event density
start_time = tcp_state_df['ts_uptime_us'].min()
tcp_state_df_time = tcp_state_df.with_columns([
    ((pl.col('ts_uptime_us') - start_time) / 1_000_000).alias('time_seconds')
])

# Create histogram of events over time
time_bins = np.linspace(0, tcp_state_df_time['time_seconds'].max(), 50)
transitions_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'TRANSITION')['time_seconds'].to_numpy()
processing_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'PROCESSING')['time_seconds'].to_numpy()
errors_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'ERROR')['time_seconds'].to_numpy()

ax2.hist([transitions_time, processing_time, errors_time], 
         bins=time_bins, label=['Transitions', 'Processing', 'Errors'], 
         stacked=True, alpha=0.7)
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Event Count')
ax2.set_title('TCP State Events Over Time')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Cell 6: Correlate with tcp_v4_rcv data
print("\n" + "="*50)
print("CORRELATION WITH TCP_V4_RCV BRANCH DATA")
print("="*50)

# Find overlapping time windows
tcp_v4_time_range = (tcp_df['ts_uptime_us'].min(), tcp_df['ts_uptime_us'].max())
tcp_state_time_range = (tcp_state_df['ts_uptime_us'].min(), tcp_state_df['ts_uptime_us'].max())

overlap_start = max(tcp_v4_time_range[0], tcp_state_time_range[0])
overlap_end = min(tcp_v4_time_range[1], tcp_state_time_range[1])

if overlap_start < overlap_end:
    print(f"Time overlap: {(overlap_end - overlap_start) / 1_000_000:.2f} seconds")
    
    # Filter both datasets to overlapping time
    tcp_v4_overlap = tcp_df.filter(
        (pl.col('ts_uptime_us') >= overlap_start) & 
        (pl.col('ts_uptime_us') <= overlap_end)
    )
    tcp_state_overlap = tcp_state_df.filter(
        (pl.col('ts_uptime_us') >= overlap_start) & 
        (pl.col('ts_uptime_us') <= overlap_end)
    )
    
    # Analyze patterns
    print(f"\nDuring overlap period:")
    print(f"- TCP v4 receive events: {len(tcp_v4_overlap):,}")
    print(f"- TCP state events: {len(tcp_state_overlap):,}")
    
    # Show branch distribution during state transitions
    print("\nTCP v4 branches during state processing:")
    branch_dist = tcp_v4_overlap.group_by('branch_name').len().sort('len', descending=True)
    for row in branch_dist.head(10).iter_rows():
        print(f"  {row[0]:25}: {row[1]:6,} ({row[1]/len(tcp_v4_overlap)*100:5.1f}%)")

# Cell 7: Performance insights
print("\n" + "="*50)
print("PERFORMANCE INSIGHTS")
print("="*50)

# Calculate metrics
total_time = (tcp_state_df['ts_uptime_us'].max() - tcp_state_df['ts_uptime_us'].min()) / 1_000_000
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
errors = tcp_state_df.filter(pl.col('event_type_name') == 'ERROR')

# Connection establishment rate
new_connections = transitions.filter(pl.col('new_state_name') == 'ESTABLISHED')
connection_rate = len(new_connections) / total_time if total_time > 0 else 0

# Error rate
error_rate = len(errors) / len(tcp_state_df) * 100 if len(tcp_state_df) > 0 else 0

# State distribution
state_dist = tcp_state_df.filter(pl.col('event_type_name') == 'PROCESSING').group_by('new_state_name').len()

print(f"Collection duration: {total_time:.2f} seconds")
print(f"Connection establishment rate: {connection_rate:.1f} connections/second")
print(f"Error rate: {error_rate:.2f}%")
print(f"Total state transitions: {len(transitions):,}")

print("\nState distribution (PROCESSING events):")
for row in state_dist.sort('len', descending=True).iter_rows():
    print(f"  {row[0]:15}: {row[1]:6,} events")

# Identify potential issues
print("\nPotential Issues:")
if error_rate > 1:
    print(f"⚠️  High error rate detected: {error_rate:.2f}%")
    
challenge_acks = tcp_state_df.filter(pl.col('event_subtype_name') == 'CHALLENGE_ACK')
if len(challenge_acks) > 0:
    print(f"⚠️  Challenge ACKs detected: {len(challenge_acks)} (possible SYN flood)")
    
resets = tcp_state_df.filter(pl.col('event_subtype_name') == 'RESET')
if len(resets) > 0:
    print(f"⚠️  Connection resets: {len(resets)}")

# Check for connection leaks
time_wait_trans = transitions.filter(pl.col('new_state_name') == 'TIME_WAIT')
close_trans = transitions.filter(pl.col('new_state_name').str.contains('CLOSE'))
if len(new_connections) > 0:
    close_ratio = (len(time_wait_trans) + len(close_trans)) / len(new_connections)
    if close_ratio < 0.8:
        print(f"⚠️  Possible connection leak: only {close_ratio*100:.1f}% of connections properly closed")

In [ ]:
collect = Collector("./config/redis_never.yaml")
collect.start_collection(None)
data = collect.stop_collection()

In [ ]:
# Analyze TCP branches
import polars as pl
tcp_df = pl.read_parquet(data["tcp_v4_rcv"][0])

print(tcp_df.group_by("comm").count().sort("count", descending=True))

redis = tcp_df.filter(pl.col("comm").str.contains("redis-server"))

# Show branch distribution
print(redis.group_by("branch_name").count().sort("count", descending=True))

# Show drop reasons
drops = redis.filter(pl.col("drop_reason") > 0)
print(drops.group_by("drop_reason_name").count())

In [ ]:
tcp_state_df = pl.read_parquet(data["tcp_state_process"][0])

print(f"Loaded {len(tcp_df):,} tcp_v4_rcv events")
print(f"Loaded {len(tcp_state_df):,} tcp_state_process events")

# Cell 2: Basic exploration of TCP state data
# Show sample of state transitions
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
print("\nSample State Transitions:")
print(transitions.select(['comm', 'old_state_name', 'new_state_name', 'ts_uptime_us']).head(10))

# Show event type distribution
print("\nEvent Type Distribution:")
print(tcp_state_df.group_by('event_type_name').len().sort('len', descending=True))

# Cell 3: Deep dive into specific process (e.g., nc or iperf3)
# Filter for a specific process
process_name = "nc"  # or "iperf3" depending on your test
process_events = tcp_state_df.filter(pl.col('comm').str.contains(process_name))

if len(process_events) > 0:
    print(f"\n{process_name} TCP State Analysis:")
    print(f"Total events: {len(process_events)}")
    
    # Show state progression
    print("\nState progression timeline:")
    timeline = process_events.select(['ts_uptime_us', 'event_type_name', 'old_state_name', 'new_state_name']).sort('ts_uptime_us')
    
    # Convert to relative time in milliseconds
    start_time = timeline['ts_uptime_us'].min()
    timeline_with_relative = timeline.with_columns([
        ((pl.col('ts_uptime_us') - start_time) / 1000).alias('relative_ms')
    ])
    
    # Show key events
    for row in timeline_with_relative.head(20).iter_rows():
        if row[1] == 'TRANSITION':
            print(f"{row[4]:8.1f}ms: {row[2]} → {row[3]}")
        else:
            print(f"{row[4]:8.1f}ms: {row[1]} in {row[3]}")

# Cell 4: Connection lifecycle analysis
print("\n" + "="*50)
print("CONNECTION LIFECYCLE ANALYSIS")
print("="*50)

# Group events by process and analyze connection patterns
connection_patterns = tcp_state_df.group_by('comm').agg([
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'SYN_SENT').len().alias('syn_sent_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'SYN_RECV').len().alias('syn_recv_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'ESTABLISHED').len().alias('established_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'TIME_WAIT').len().alias('time_wait_count'),
    pl.col('event_type_name').filter(pl.col('event_type_name') == 'ERROR').len().alias('error_count')
]).filter(pl.col('established_count') > 0).sort('established_count', descending=True)

print("Connection patterns by process:")
print(connection_patterns.head(10))

# Cell 5: Visualize state machine flow
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# State transition graph
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
if len(transitions) > 0:
    # Count transitions
    trans_counts = transitions.group_by(['old_state_name', 'new_state_name']).len().sort('len', descending=True)
    
    # Create a simple flow diagram representation
    states = set()
    for row in trans_counts.iter_rows():
        states.add(row[0])
        states.add(row[1])
    
    # Plot top transitions as a bar chart
    trans_labels = [f"{row[0]} → {row[1]}" for row in trans_counts.head(15).iter_rows()]
    trans_values = [row[2] for row in trans_counts.head(15).iter_rows()]
    
    ax1.barh(range(len(trans_labels)), trans_values)
    ax1.set_yticks(range(len(trans_labels)))
    ax1.set_yticklabels(trans_labels)
    ax1.set_xlabel('Count')
    ax1.set_title('Top 15 State Transitions')
    ax1.grid(axis='x', alpha=0.3)

# Event timeline
# Create time-based event density
start_time = tcp_state_df['ts_uptime_us'].min()
tcp_state_df_time = tcp_state_df.with_columns([
    ((pl.col('ts_uptime_us') - start_time) / 1_000_000).alias('time_seconds')
])

# Create histogram of events over time
time_bins = np.linspace(0, tcp_state_df_time['time_seconds'].max(), 50)
transitions_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'TRANSITION')['time_seconds'].to_numpy()
processing_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'PROCESSING')['time_seconds'].to_numpy()
errors_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'ERROR')['time_seconds'].to_numpy()

ax2.hist([transitions_time, processing_time, errors_time], 
         bins=time_bins, label=['Transitions', 'Processing', 'Errors'], 
         stacked=True, alpha=0.7)
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Event Count')
ax2.set_title('TCP State Events Over Time')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Cell 6: Correlate with tcp_v4_rcv data
print("\n" + "="*50)
print("CORRELATION WITH TCP_V4_RCV BRANCH DATA")
print("="*50)

# Find overlapping time windows
tcp_v4_time_range = (tcp_df['ts_uptime_us'].min(), tcp_df['ts_uptime_us'].max())
tcp_state_time_range = (tcp_state_df['ts_uptime_us'].min(), tcp_state_df['ts_uptime_us'].max())

overlap_start = max(tcp_v4_time_range[0], tcp_state_time_range[0])
overlap_end = min(tcp_v4_time_range[1], tcp_state_time_range[1])

if overlap_start < overlap_end:
    print(f"Time overlap: {(overlap_end - overlap_start) / 1_000_000:.2f} seconds")
    
    # Filter both datasets to overlapping time
    tcp_v4_overlap = tcp_df.filter(
        (pl.col('ts_uptime_us') >= overlap_start) & 
        (pl.col('ts_uptime_us') <= overlap_end)
    )
    tcp_state_overlap = tcp_state_df.filter(
        (pl.col('ts_uptime_us') >= overlap_start) & 
        (pl.col('ts_uptime_us') <= overlap_end)
    )
    
    # Analyze patterns
    print(f"\nDuring overlap period:")
    print(f"- TCP v4 receive events: {len(tcp_v4_overlap):,}")
    print(f"- TCP state events: {len(tcp_state_overlap):,}")
    
    # Show branch distribution during state transitions
    print("\nTCP v4 branches during state processing:")
    branch_dist = tcp_v4_overlap.group_by('branch_name').len().sort('len', descending=True)
    for row in branch_dist.head(10).iter_rows():
        print(f"  {row[0]:25}: {row[1]:6,} ({row[1]/len(tcp_v4_overlap)*100:5.1f}%)")

# Cell 7: Performance insights
print("\n" + "="*50)
print("PERFORMANCE INSIGHTS")
print("="*50)

# Calculate metrics
total_time = (tcp_state_df['ts_uptime_us'].max() - tcp_state_df['ts_uptime_us'].min()) / 1_000_000
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
errors = tcp_state_df.filter(pl.col('event_type_name') == 'ERROR')

# Connection establishment rate
new_connections = transitions.filter(pl.col('new_state_name') == 'ESTABLISHED')
connection_rate = len(new_connections) / total_time if total_time > 0 else 0

# Error rate
error_rate = len(errors) / len(tcp_state_df) * 100 if len(tcp_state_df) > 0 else 0

# State distribution
state_dist = tcp_state_df.filter(pl.col('event_type_name') == 'PROCESSING').group_by('new_state_name').len()

print(f"Collection duration: {total_time:.2f} seconds")
print(f"Connection establishment rate: {connection_rate:.1f} connections/second")
print(f"Error rate: {error_rate:.2f}%")
print(f"Total state transitions: {len(transitions):,}")

print("\nState distribution (PROCESSING events):")
for row in state_dist.sort('len', descending=True).iter_rows():
    print(f"  {row[0]:15}: {row[1]:6,} events")

# Identify potential issues
print("\nPotential Issues:")
if error_rate > 1:
    print(f"⚠️  High error rate detected: {error_rate:.2f}%")
    
challenge_acks = tcp_state_df.filter(pl.col('event_subtype_name') == 'CHALLENGE_ACK')
if len(challenge_acks) > 0:
    print(f"⚠️  Challenge ACKs detected: {len(challenge_acks)} (possible SYN flood)")
    
resets = tcp_state_df.filter(pl.col('event_subtype_name') == 'RESET')
if len(resets) > 0:
    print(f"⚠️  Connection resets: {len(resets)}")

# Check for connection leaks
time_wait_trans = transitions.filter(pl.col('new_state_name') == 'TIME_WAIT')
close_trans = transitions.filter(pl.col('new_state_name').str.contains('CLOSE'))
if len(new_connections) > 0:
    close_ratio = (len(time_wait_trans) + len(close_trans)) / len(new_connections)
    if close_ratio < 0.8:
        print(f"⚠️  Possible connection leak: only {close_ratio*100:.1f}% of connections properly closed")

In [ ]:
# Create collector with XSBench configuration
collect = Collector("./config/xsbench.yaml")

# Start collection and run XSBench
print("Starting collection with XSBench workload...")
collect.start_collection(None)

# Wait for XSBench to complete
print("Running XSBench benchmark...")
data = collect.stop_collection()




In [ ]:
# Analyze TCP traffic generated by XSBench
print("\nAnalyzing TCP traffic from XSBench:")
tcp_df = pl.read_parquet(data["tcp_v4_rcv"][0])

# Show branch distribution
print("\nTCP branch distribution:")
print(tcp_df.group_by("branch_name").len().sort("len", descending=True))

# Show drop reasons if any
drops = tcp_df.filter(pl.col("drop_reason") > 0)
if len(drops) > 0:
    print("\nDropped packets:")
    print(drops.group_by("drop_reason_name").len())
else:
    print("\nNo dropped packets detected")

# Show process-specific TCP activity
print("\nTCP activity by process:")
process_tcp = tcp_df.group_by("comm").len().sort("len", descending=True).head(10)
print(process_tcp)

# Check for XSBench-specific activity
xsbench_traffic = tcp_df.filter(pl.col("comm").str.contains("XSBench"))
if len(xsbench_traffic) > 0:
    print(f"\nXSBench generated {len(xsbench_traffic)} TCP events")
else:
    print("\nNo direct TCP traffic from XSBench process detected")

In [ ]:
tcp_state_df = pl.read_parquet(data["tcp_state_process"][0])

print(f"Loaded {len(tcp_df):,} tcp_v4_rcv events")
print(f"Loaded {len(tcp_state_df):,} tcp_state_process events")

# Cell 2: Basic exploration of TCP state data
# Show sample of state transitions
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
print("\nSample State Transitions:")
print(transitions.select(['comm', 'old_state_name', 'new_state_name', 'ts_uptime_us']).head(10))

# Show event type distribution
print("\nEvent Type Distribution:")
print(tcp_state_df.group_by('event_type_name').len().sort('len', descending=True))

# Cell 3: Deep dive into specific process (e.g., nc or iperf3)
# Filter for a specific process
process_name = "nc"  # or "iperf3" depending on your test
process_events = tcp_state_df.filter(pl.col('comm').str.contains(process_name))

if len(process_events) > 0:
    print(f"\n{process_name} TCP State Analysis:")
    print(f"Total events: {len(process_events)}")
    
    # Show state progression
    print("\nState progression timeline:")
    timeline = process_events.select(['ts_uptime_us', 'event_type_name', 'old_state_name', 'new_state_name']).sort('ts_uptime_us')
    
    # Convert to relative time in milliseconds
    start_time = timeline['ts_uptime_us'].min()
    timeline_with_relative = timeline.with_columns([
        ((pl.col('ts_uptime_us') - start_time) / 1000).alias('relative_ms')
    ])
    
    # Show key events
    for row in timeline_with_relative.head(20).iter_rows():
        if row[1] == 'TRANSITION':
            print(f"{row[4]:8.1f}ms: {row[2]} → {row[3]}")
        else:
            print(f"{row[4]:8.1f}ms: {row[1]} in {row[3]}")

# Cell 4: Connection lifecycle analysis
print("\n" + "="*50)
print("CONNECTION LIFECYCLE ANALYSIS")
print("="*50)

# Group events by process and analyze connection patterns
connection_patterns = tcp_state_df.group_by('comm').agg([
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'SYN_SENT').len().alias('syn_sent_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'SYN_RECV').len().alias('syn_recv_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'ESTABLISHED').len().alias('established_count'),
    pl.col('event_type_name').filter(pl.col('new_state_name') == 'TIME_WAIT').len().alias('time_wait_count'),
    pl.col('event_type_name').filter(pl.col('event_type_name') == 'ERROR').len().alias('error_count')
]).filter(pl.col('established_count') > 0).sort('established_count', descending=True)

print("Connection patterns by process:")
print(connection_patterns.head(10))

# Cell 5: Visualize state machine flow
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# State transition graph
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
if len(transitions) > 0:
    # Count transitions
    trans_counts = transitions.group_by(['old_state_name', 'new_state_name']).len().sort('len', descending=True)
    
    # Create a simple flow diagram representation
    states = set()
    for row in trans_counts.iter_rows():
        states.add(row[0])
        states.add(row[1])
    
    # Plot top transitions as a bar chart
    trans_labels = [f"{row[0]} → {row[1]}" for row in trans_counts.head(15).iter_rows()]
    trans_values = [row[2] for row in trans_counts.head(15).iter_rows()]
    
    ax1.barh(range(len(trans_labels)), trans_values)
    ax1.set_yticks(range(len(trans_labels)))
    ax1.set_yticklabels(trans_labels)
    ax1.set_xlabel('Count')
    ax1.set_title('Top 15 State Transitions')
    ax1.grid(axis='x', alpha=0.3)

# Event timeline
# Create time-based event density
start_time = tcp_state_df['ts_uptime_us'].min()
tcp_state_df_time = tcp_state_df.with_columns([
    ((pl.col('ts_uptime_us') - start_time) / 1_000_000).alias('time_seconds')
])

# Create histogram of events over time
time_bins = np.linspace(0, tcp_state_df_time['time_seconds'].max(), 50)
transitions_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'TRANSITION')['time_seconds'].to_numpy()
processing_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'PROCESSING')['time_seconds'].to_numpy()
errors_time = tcp_state_df_time.filter(pl.col('event_type_name') == 'ERROR')['time_seconds'].to_numpy()

ax2.hist([transitions_time, processing_time, errors_time], 
         bins=time_bins, label=['Transitions', 'Processing', 'Errors'], 
         stacked=True, alpha=0.7)
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Event Count')
ax2.set_title('TCP State Events Over Time')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Cell 6: Correlate with tcp_v4_rcv data
print("\n" + "="*50)
print("CORRELATION WITH TCP_V4_RCV BRANCH DATA")
print("="*50)

# Find overlapping time windows
tcp_v4_time_range = (tcp_df['ts_uptime_us'].min(), tcp_df['ts_uptime_us'].max())
tcp_state_time_range = (tcp_state_df['ts_uptime_us'].min(), tcp_state_df['ts_uptime_us'].max())

overlap_start = max(tcp_v4_time_range[0], tcp_state_time_range[0])
overlap_end = min(tcp_v4_time_range[1], tcp_state_time_range[1])

if overlap_start < overlap_end:
    print(f"Time overlap: {(overlap_end - overlap_start) / 1_000_000:.2f} seconds")
    
    # Filter both datasets to overlapping time
    tcp_v4_overlap = tcp_df.filter(
        (pl.col('ts_uptime_us') >= overlap_start) & 
        (pl.col('ts_uptime_us') <= overlap_end)
    )
    tcp_state_overlap = tcp_state_df.filter(
        (pl.col('ts_uptime_us') >= overlap_start) & 
        (pl.col('ts_uptime_us') <= overlap_end)
    )
    
    # Analyze patterns
    print(f"\nDuring overlap period:")
    print(f"- TCP v4 receive events: {len(tcp_v4_overlap):,}")
    print(f"- TCP state events: {len(tcp_state_overlap):,}")
    
    # Show branch distribution during state transitions
    print("\nTCP v4 branches during state processing:")
    branch_dist = tcp_v4_overlap.group_by('branch_name').len().sort('len', descending=True)
    for row in branch_dist.head(10).iter_rows():
        print(f"  {row[0]:25}: {row[1]:6,} ({row[1]/len(tcp_v4_overlap)*100:5.1f}%)")

# Cell 7: Performance insights
print("\n" + "="*50)
print("PERFORMANCE INSIGHTS")
print("="*50)

# Calculate metrics
total_time = (tcp_state_df['ts_uptime_us'].max() - tcp_state_df['ts_uptime_us'].min()) / 1_000_000
transitions = tcp_state_df.filter(pl.col('event_type_name') == 'TRANSITION')
errors = tcp_state_df.filter(pl.col('event_type_name') == 'ERROR')

# Connection establishment rate
new_connections = transitions.filter(pl.col('new_state_name') == 'ESTABLISHED')
connection_rate = len(new_connections) / total_time if total_time > 0 else 0

# Error rate
error_rate = len(errors) / len(tcp_state_df) * 100 if len(tcp_state_df) > 0 else 0

# State distribution
state_dist = tcp_state_df.filter(pl.col('event_type_name') == 'PROCESSING').group_by('new_state_name').len()

print(f"Collection duration: {total_time:.2f} seconds")
print(f"Connection establishment rate: {connection_rate:.1f} connections/second")
print(f"Error rate: {error_rate:.2f}%")
print(f"Total state transitions: {len(transitions):,}")

print("\nState distribution (PROCESSING events):")
for row in state_dist.sort('len', descending=True).iter_rows():
    print(f"  {row[0]:15}: {row[1]:6,} events")

# Identify potential issues
print("\nPotential Issues:")
if error_rate > 1:
    print(f"⚠️  High error rate detected: {error_rate:.2f}%")
    
challenge_acks = tcp_state_df.filter(pl.col('event_subtype_name') == 'CHALLENGE_ACK')
if len(challenge_acks) > 0:
    print(f"⚠️  Challenge ACKs detected: {len(challenge_acks)} (possible SYN flood)")
    
resets = tcp_state_df.filter(pl.col('event_subtype_name') == 'RESET')
if len(resets) > 0:
    print(f"⚠️  Connection resets: {len(resets)}")

# Check for connection leaks
time_wait_trans = transitions.filter(pl.col('new_state_name') == 'TIME_WAIT')
close_trans = transitions.filter(pl.col('new_state_name').str.contains('CLOSE'))
if len(new_connections) > 0:
    close_ratio = (len(time_wait_trans) + len(close_trans)) / len(new_connections)
    if close_ratio < 0.8:
        print(f"⚠️  Possible connection leak: only {close_ratio*100:.1f}% of connections properly closed")